<a href="https://colab.research.google.com/github/Oppepper97/AI_assignment/blob/main/AI_Assignment_draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Predicting the FIFA 2022 World Cup**

Welcome to this Jupyter Notebook for the AI Assignment of Pepijn van Vugt (1735664).

In this Jupyter Notebook we will be building a neural network to predict the outcomes of the 2022 FIFA World Cup (WC). This is a highly anticipated event took  take place in Qatar from November 21st to December 18th, 2022, and will feature 32 national teams competing for the trophy.

Predicting the outcomes of sports events is a challenging task, as it involves modeling the performance of teams and players, as well as accounting for various factors such as injuries, form, tactics, and even luck. However, machine learning techniques, and particularly deep learning models, have shown great potential in this domain, and have been used to predict the outcomes of various competitions, including soccer matches.

In this notebook, we will start by exploring the data that we will use to train our model. This will include information about the teams, the goals scored and conceded by all the participating teams, and the matches they have played. We will then pre-process and clean the data, and finally, we will build and train a neural network to make predictions.


# Gathering the data through scraping

In [1]:
# Firstly, I import the right packages to scrape all the World Cup matches.
import pandas as pd
from bs4 import BeautifulSoup
import requests

The following code is from:

Andrade, F. (2022a, November 12). Web Scraping Football Matches From The World Cups 1930 to 2022 with Python. Medium. https://medium.com/geekculture/web-scraping-football-matches-from-the-world-cups-1930-to-2022-with-python-d2a1d578f034

In [2]:
years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974,
         1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014,
         2018]


def get_matches(year):
    web = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'
    response = requests.get(web)
    content = response.text
    soup = BeautifulSoup(content, 'lxml')
    matches = soup.find_all('div', class_='footballbox')

    home = []
    score = []
    away = []

    for match in matches:
        home.append(match.find('th', class_='fhome').get_text())
        score.append(match.find('th', class_='fscore').get_text())
        away.append(match.find('th', class_='faway').get_text())

    dict_football = {'home': home, 'score': score, 'away': away}
    df_football = pd.DataFrame(dict_football)
    df_football['year'] = year
    return df_football


# results: historical data
fifa = [get_matches(year) for year in years]
df_fifa = pd.concat(fifa, ignore_index=True)
df_fifa.to_csv("fifa_worldcup_historical_data.csv", index=False)

# fixture
df_fixture = get_matches(2022)
df_fixture.to_csv('fifa_worldcup_fixture.csv', index=False)

Below you can see a table with matchdata from the first World Cup match between 
France and Mexico in 1930, and the final of the 2018 World Cup between France and Croatia. 

####Note:
For some reason the dataframe shows 601 rows, while it should contain 458 rows. That's why I import the CSV File and will change it into a dataframe.

In [3]:
from google.colab import files
uploaded = files.upload()
# Here I import the dataset with WC matches from 1930 till 2018

Saving fifa_worldcup_matches.csv to fifa_worldcup_matches.csv


In [4]:
df_fifa = pd.read_csv('fifa_worldcup_matches.csv')
print(df_fifa)

           home         score      away  year
0       France            4–1    Mexico  1930
1    Argentina            1–0    France  1930
2        Chile            3–0    Mexico  1930
3        Chile            1–0    France  1930
4    Argentina            6–3    Mexico  1930
..          ...           ...       ...   ...
452     Russia   2–2 (a.e.t.)   Croatia  2018
453     France            1–0   Belgium  2018
454    Croatia   2–1 (a.e.t.)   England  2018
455    Belgium            2–0   England  2018
456     France            4–2   Croatia  2018

[457 rows x 4 columns]


In the following table you can see the matches from the 2022 World Cup. However, the scraped data already includes the score from all the 64 matches played during the WC, because the WC has already been finished when coding this assignment. Predicting won't make sense if I already have the scores, because this will influence the model. So, I'll remove the column 'score' to make the model more accurate based on data which was available before the WC started.

In [5]:
df_fixture = df_fixture.drop('score', axis=1)
df_fixture

,home,away,year
0,Qatar,Ecuador,2022
1,Senegal,Netherlands,2022
2,Qatar,Senegal,2022
3,Netherlands,Ecuador,2022
4,Ecuador,Senegal,2022
...,...,...,...
59,England,France,2022
60,Argentina,Croatia,2022
61,France,Morocco,2022
62,Croatia,Morocco,2022


In [6]:
df_fixture.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   home    64 non-null     object
 1   away    64 non-null     object
 2   year    64 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 1.6+ KB


# Gathering tables

To predict the outcomes of the 2022 WC, I also need to get all the tables from the 2018 WC. I can extract these from the wikipedia website. I used a website called the 'waybackmachine' which takes snapshots of earlier versions of websites. 

I chose this method, because at the moment of doing this assignment,the World Cup has already been played. If I used the current website, all the group tables would've already contained data. In this way I have empty group tables. Otherwise I had to clean all the group tables to get rid of the data and strings inside.

In [7]:
WC_tables = pd.read_html('https://web.archive.org/web/20221120065417/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')

Here I will create a loop that iterates over the extracted tables and shows where in the page the table is located:

In [8]:
for i in range(14,70,7):
    print(i)
    df = WC_tables[i]

14
21
28
35
42
49
56
63


To get all the tables, I have to use the new dictionary called 'WC_tables', and use elements such as '0', '1', '2' to find the right element which contain the all the WC groups (Group A until Group H). 
All the group tables occur in ever 7 steps, starting from 14. Which means:

- Group A is in table 14
- Group B is in table 21
- Group C is in table 28
- Group D is in table 35
- Group E is in table 42
- Group F is in table 49
- Group G is in table 56
- Group H is in table 63

In [9]:
WC_tables[14] # This is group A of the World Cup

,Pos,"Team.mw-parser-output .navbar{display:inline;font-size:88%;font-weight:normal}.mw-parser-output .navbar-collapse{float:left;text-align:left}.mw-parser-output .navbar-boxtext{word-spacing:0}.mw-parser-output .navbar ul{display:inline-block;white-space:nowrap;line-height:inherit}.mw-parser-output .navbar-brackets::before{margin-right:-0.125em;content:""[ ""}.mw-parser-output .navbar-brackets::after{margin-left:-0.125em;content:"" ]""}.mw-parser-output .navbar li{word-spacing:-0.125em}.mw-parser-output .navbar a>span,.mw-parser-output .navbar a>abbr{text-decoration:inherit}.mw-parser-output .navbar-mini abbr{font-variant:small-caps;border-bottom:none;text-decoration:none;cursor:inherit}.mw-parser-output .navbar-ct-full{font-size:114%;margin:0 7em}.mw-parser-output .navbar-ct-mini{font-size:114%;margin:0 4em}vte",Pld,W,D,L,GF,GA,GD,Pts,Qualification
0,1,Qatar (H),0,0,0,0,0,0,0,0,Advance to knockout stage
1,2,Ecuador,0,0,0,0,0,0,0,0,Advance to knockout stage
2,3,Senegal,0,0,0,0,0,0,0,0,NaN
3,4,Netherlands,0,0,0,0,0,0,0,0,NaN


For the following part I will need to import the alphabet to connect the names of the groups (Group A, B etc) to the table numbers (14,21 etc).

In [10]:
from string import ascii_uppercase as alphabet

In [11]:
alphabet

'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [12]:
WC_tables = pd.read_html('https://web.archive.org/web/20221120065417/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')

dict_table = {}
for letter, i in zip(alphabet, range(14,70,7)):
    df = WC_tables[i]
    df.rename(columns={df.columns[1]:'Team'}, inplace=True)
    dict_table[f'Group {letter}'] = df

In [13]:
dict_table.keys()

dict_keys(['Group A', 'Group B', 'Group C', 'Group D', 'Group E', 'Group F', 'Group G', 'Group H'])

Here you can see the final result of the cleaned groups which are lists now. 

In [14]:
dict_table['Group A']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts,Qualification
0,1,Qatar (H),0,0,0,0,0,0,0,0,Advance to knockout stage
1,2,Ecuador,0,0,0,0,0,0,0,0,Advance to knockout stage
2,3,Senegal,0,0,0,0,0,0,0,0,NaN
3,4,Netherlands,0,0,0,0,0,0,0,0,NaN


# Cleaning
The CSV Files I have saved have some inconsistencies because they were scraped from Wikipedia. If I look at the CSV File in Excel, I can see that the 1970 World Cup only has only 7 stored matches. That it's not correct. I downloaded the missing dataset from [this](https://github.com/ifrankandrade/fifa-world-cup-2022-prediction/blob/main/data/fifa_worldcup_missing_data.csv) github page.

For this I need to upload the file to Google Colab through the import files function of Google Colab.


In [15]:
from google.colab import files
uploaded = files.upload()
# Here I import the 'fifa world cup missing dataset

Saving fifa_missing_data.csv to fifa_missing_data.csv


In [16]:
import pandas as pd

In [17]:
df_missing_data = pd.read_csv('fifa_missing_data.csv')
print(df_missing_data)

              home score           away  year
0          Mexico    0–0   Soviet Union  1970
1         Belgium    3–0    El Salvador  1970
2    Soviet Union    4–1        Belgium  1970
3          Mexico    4–0    El Salvador  1970
4    Soviet Union    2–0    El Salvador  1970
..             ...   ...            ...   ...
503         Spain    0–1    Switzerland  2010
504         Chile    1–0    Switzerland  2010
505         Spain    2–0       Honduras  2010
506         Chile    1–2          Spain  2010
507   Switzerland    0–0       Honduras  2010

[508 rows x 4 columns]



I will merge the dataframe with historical data with the missing data dataframe to get a full dataframe that contains all the matches and data from 1930 until 2018. This is needed to calculate the goals scored and goals conceded for each team in all the played World Cups since 1930.

In [18]:
df_missing_data.dropna(inplace=True)
df_fifa_all_worldcup = pd.concat([df_fifa, df_missing_data], ignore_index=True)
df_fifa_all_worldcup.drop_duplicates(inplace=True)
df_fifa_all_worldcup.sort_values('year', inplace=True)
#df_fifa_all_worldcup.drop(757)
df_fifa_all_worldcup

,home,score,away,year
0,France,4–1,Mexico,1930
17,Uruguay,4–2,Argentina,1930
16,Uruguay,6–1,Yugoslavia,1930
15,Argentina,6–1,United States,1930
14,Paraguay,1–0,Belgium,1930
...,...,...,...,...
419,Brazil,2–0,Costa Rica,2018
420,Serbia,1–2,Switzerland,2018
421,Serbia,0–2,Brazil,2018
408,France,1–0,Peru,2018


In [19]:
from google.colab import files
df_fifa_all_worldcup.to_csv('clean_fifa_worldcup.csv',index=False)

The first World Cup was a small one with only 18 matches. The tournament became more popular and the sport was developing itself more and more. Every edition more countries were participating, and thus more matches were being played. Since 1998 matches are being played. In 2026, 48 different countries will participate in the World Cup, making it the biggest tournament yet.

Here you can see the amount of matches being played per World Cup:

In [20]:
years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974,
         1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014,
         2018]

for year in years:
    print(year, len(df_fifa_all_worldcup[df_fifa_all_worldcup['year']==year]))

1930 18
1934 17
1938 19
1950 22
1954 26
1958 35
1962 32
1966 32
1970 32
1974 38
1978 38
1982 52
1986 52
1990 52
1994 52
1998 64
2002 64
2006 64
2010 64
2014 64
2018 64


# Calculating the historical results of all the World Cup matches from 1930 till 2018
To predict the outcome of the 2022 World Cup I will have to calculate the strength of all the participating teams. I will do this by calculating the goals scored and goals conceded in a historical way. This will result in historical records up until the time a team will play their next match. I will use the mean to get this record.

Now that the whole df was cleaned. I could take the sum of 'HomeGoals' and 'AwayGoals' and add these to a new column named 'TotalGoals'. After this step I can calculate the historical results and team strength.

In [21]:
# deleting match with walk over
delete_index = df_fifa_all_worldcup[df_fifa_all_worldcup['home'].str.contains('Sweden') &
                                    df_fifa_all_worldcup['away'].str.contains('Austria')].index

df_fifa_all_worldcup.drop(index=delete_index, inplace=True)

# cleanning score and home/away columns
df_fifa_all_worldcup['score'] = df_fifa_all_worldcup['score'].str.replace('[^\d–]', '', regex=True)
df_fifa_all_worldcup['home'] = df_fifa_all_worldcup['home'].str.strip() # clean blank spaces: Yugoslavia twice
df_fifa_all_worldcup['away'] = df_fifa_all_worldcup['away'].str.strip()

# splitting score columns into home and away goals and dropping score column
df_fifa_all_worldcup[['HomeGoals', 'AwayGoals']] = df_fifa_all_worldcup['score'].str.split('–', expand=True)
df_fifa_all_worldcup.drop('score', axis=1, inplace=True)

# renaming columns and changing data types
df_fifa_all_worldcup.rename(columns={'home': 'HomeTeam', 'away': 'AwayTeam', 
                                   'year':'Year'}, inplace=True)
df_fifa_all_worldcup = df_fifa_all_worldcup.astype({'HomeGoals': int, 'AwayGoals':int, 'Year': int})

# creating new column "totalgoals"
df_fifa_all_worldcup['TotalGoals'] = df_fifa_all_worldcup['HomeGoals'] + df_fifa_all_worldcup['AwayGoals']
df_fifa_all_worldcup

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
17,Uruguay,Argentina,1930,4,2,6
16,Uruguay,Yugoslavia,1930,6,1,7
15,Argentina,United States,1930,6,1,7
14,Paraguay,Belgium,1930,1,0,1
...,...,...,...,...,...,...
419,Brazil,Costa Rica,2018,2,0,2
420,Serbia,Switzerland,2018,1,2,3
421,Serbia,Brazil,2018,0,2,2
408,France,Peru,2018,1,0,1


In [22]:
df_worldcup = df_fifa_all_worldcup.rename(columns={'score': 'result'})
df_worldcup

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
17,Uruguay,Argentina,1930,4,2,6
16,Uruguay,Yugoslavia,1930,6,1,7
15,Argentina,United States,1930,6,1,7
14,Paraguay,Belgium,1930,1,0,1
...,...,...,...,...,...,...
419,Brazil,Costa Rica,2018,2,0,2
420,Serbia,Switzerland,2018,1,2,3
421,Serbia,Brazil,2018,0,2,2
408,France,Peru,2018,1,0,1


In [23]:
df_worldcup

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
17,Uruguay,Argentina,1930,4,2,6
16,Uruguay,Yugoslavia,1930,6,1,7
15,Argentina,United States,1930,6,1,7
14,Paraguay,Belgium,1930,1,0,1
...,...,...,...,...,...,...
419,Brazil,Costa Rica,2018,2,0,2
420,Serbia,Switzerland,2018,1,2,3
421,Serbia,Brazil,2018,0,2,2
408,France,Peru,2018,1,0,1


To rename the columns and calculate the mean goals scored and conceded later, I need to split the dataframe into a home and away dateframe.

In [24]:
df_home = df_worldcup[['Year','HomeTeam', 'HomeGoals', 'AwayGoals']]
df_away = df_worldcup[['Year','AwayTeam', 'HomeGoals', 'AwayGoals']]

In [25]:
df_home

,Year,HomeTeam,HomeGoals,AwayGoals
0,1930,France,4,1
17,1930,Uruguay,4,2
16,1930,Uruguay,6,1
15,1930,Argentina,6,1
14,1930,Paraguay,1,0
...,...,...,...,...
419,2018,Brazil,2,0
420,2018,Serbia,1,2
421,2018,Serbia,0,2
408,2018,France,1,0


In [26]:
df_away

,Year,AwayTeam,HomeGoals,AwayGoals
0,1930,Mexico,4,1
17,1930,Argentina,4,2
16,1930,Yugoslavia,6,1
15,1930,United States,6,1
14,1930,Belgium,1,0
...,...,...,...,...
419,2018,Costa Rica,2,0
420,2018,Switzerland,1,2
421,2018,Brazil,0,2
408,2018,Peru,1,0


Changing the HomeGoals and AwayGoals columns into GoalsScored and Goalsconceded

In [27]:
df_home = df_home.rename(columns={'HomeGoals': 'HomeGoalsScored', 'AwayGoals': 'HomeGoalsConceded'})
df_away = df_away.rename(columns={'HomeGoals': 'AwayGoalsConceded', 'AwayGoals': 'AwayGoalsScored'})
df_home

,Year,HomeTeam,HomeGoalsScored,HomeGoalsConceded
0,1930,France,4,1
17,1930,Uruguay,4,2
16,1930,Uruguay,6,1
15,1930,Argentina,6,1
14,1930,Paraguay,1,0
...,...,...,...,...
419,2018,Brazil,2,0
420,2018,Serbia,1,2
421,2018,Serbia,0,2
408,2018,France,1,0


Dropping the column 'Year' in df_away, otherwise I'll have 2 columns for Year when concatenating

In [28]:
df_away.drop('Year', axis=1, inplace=True)
df_away

,AwayTeam,AwayGoalsConceded,AwayGoalsScored
0,Mexico,4,1
17,Argentina,4,2
16,Yugoslavia,6,1
15,United States,6,1
14,Belgium,1,0
...,...,...,...
419,Costa Rica,2,0
420,Switzerland,1,2
421,Brazil,0,2
408,Peru,1,0


In [29]:
df_worldcup = pd.concat([df_home, df_away], axis=1)
df_worldcup

,Year,HomeTeam,HomeGoalsScored,HomeGoalsConceded,AwayTeam,AwayGoalsConceded,AwayGoalsScored
0,1930,France,4,1,Mexico,4,1
17,1930,Uruguay,4,2,Argentina,4,2
16,1930,Uruguay,6,1,Yugoslavia,6,1
15,1930,Argentina,6,1,United States,6,1
14,1930,Paraguay,1,0,Belgium,1,0
...,...,...,...,...,...,...,...
419,2018,Brazil,2,0,Costa Rica,2,0
420,2018,Serbia,1,2,Switzerland,1,2
421,2018,Serbia,0,2,Brazil,0,2
408,2018,France,1,0,Peru,1,0


I will also create a function that determines the outcome of the match based on the perspective of the home team. The result will either be a win, a draw or a loss for the hometeam. This will be added into a new column in the dataframe.

In [30]:
def determine_outcome(row):
    if row['HomeGoalsScored'] > row['AwayGoalsScored']:
        return 'win'
    elif row['HomeGoalsScored'] < row['AwayGoalsScored']:
        return 'lose'
    else:
        return 'draw'

df_worldcup['Outcome'] = df_worldcup.apply(lambda row: determine_outcome(row), axis=1)
df_worldcup

,Year,HomeTeam,HomeGoalsScored,HomeGoalsConceded,AwayTeam,AwayGoalsConceded,AwayGoalsScored,Outcome
0,1930,France,4,1,Mexico,4,1,win
17,1930,Uruguay,4,2,Argentina,4,2,win
16,1930,Uruguay,6,1,Yugoslavia,6,1,win
15,1930,Argentina,6,1,United States,6,1,win
14,1930,Paraguay,1,0,Belgium,1,0,win
...,...,...,...,...,...,...,...,...
419,2018,Brazil,2,0,Costa Rica,2,0,win
420,2018,Serbia,1,2,Switzerland,1,2,lose
421,2018,Serbia,0,2,Brazil,0,2,lose
408,2018,France,1,0,Peru,1,0,win


In [31]:
df_worldcup.to_csv('full_wc_matches.csv')

# Calculating the mean goals scored and conceded per team up until the next match of the team.

First I have to create new columns to store the mean goals scored and mean goals conceded for the home and away team.

In [32]:
df_worldcup['HomeMeanGoalsScored'] = ''
df_worldcup['HomeMeanGoalsConceded'] = ''
df_worldcup['AwayMeanGoalsScored'] = ''
df_worldcup['AwayMeanGoalsConceded'] = ''

Just for extra 'safety', I'll strip the columns for any possible impurities

In [33]:
df_worldcup.columns = df_worldcup.columns.str.strip()

Sorting the df by year and hometeam

In [34]:
df_worldcup = df_worldcup.dropna(subset=['HomeGoalsScored', 'HomeGoalsConceded'])
df_worldcup.sort_values(by=['Year', 'HomeTeam'])

,Year,HomeTeam,HomeGoalsScored,HomeGoalsConceded,AwayTeam,AwayGoalsConceded,AwayGoalsScored,Outcome,HomeMeanGoalsScored,HomeMeanGoalsConceded,AwayMeanGoalsScored,AwayMeanGoalsConceded
15,1930,Argentina,6,1,United States,6,1,win,,,,
5,1930,Argentina,3,1,Chile,3,1,win,,,,
4,1930,Argentina,6,3,Mexico,6,3,win,,,,
1,1930,Argentina,1,0,France,1,0,win,,,,
8,1930,Brazil,4,0,Bolivia,4,0,win,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
430,2018,Tunisia,1,2,England,1,2,lose,,,,
442,2018,Uruguay,2,1,Portugal,2,1,win,,,,
449,2018,Uruguay,0,2,France,0,2,lose,,,,
396,2018,Uruguay,1,0,Saudi Arabia,1,0,win,,,,


I have a dataframe including columns for the mean goals scored and conceded for the home and away team. I can start calculating the mean for these teams. 

In [35]:
df_worldcup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 900 entries, 0 to 450
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Year                   900 non-null    int64 
 1   HomeTeam               900 non-null    object
 2   HomeGoalsScored        900 non-null    int64 
 3   HomeGoalsConceded      900 non-null    int64 
 4   AwayTeam               900 non-null    object
 5   AwayGoalsConceded      900 non-null    int64 
 6   AwayGoalsScored        900 non-null    int64 
 7   Outcome                900 non-null    object
 8   HomeMeanGoalsScored    900 non-null    object
 9   HomeMeanGoalsConceded  900 non-null    object
 10  AwayMeanGoalsScored    900 non-null    object
 11  AwayMeanGoalsConceded  900 non-null    object
dtypes: int64(5), object(7)
memory usage: 91.4+ KB


In the following cell I will calculate the mean goals scored and conceded per team. This will update itself when the next match of a team will start and finish. 

This code will update the values of the Mean Goals Columns in df_worldcup  with the values in the result DataFrame, using the HomeTeam and AwayTeam columns as the keys for the update. The update method will only update the values of the existing rows in the df_worldcup DataFrame, and will leave the other rows unchanged.

In [36]:
grouped = df_worldcup.groupby(['HomeTeam', 'AwayTeam', 'Year'])

def calc_mean_goals(group):
    home_mean_scored = group['HomeGoalsScored'].expanding().mean().iloc[-1]
    home_mean_conceded = group['HomeGoalsConceded'].expanding().mean().iloc[-1]
    away_mean_scored = group['AwayGoalsScored'].expanding().mean().iloc[-1]
    away_mean_conceded = group['AwayGoalsConceded'].expanding().mean().iloc[-1]
    return pd.Series({'HomeMeanGoalsScored': home_mean_scored, 
                      'HomeMeanGoalsConceded': home_mean_conceded,
                      'AwayMeanGoalsScored': away_mean_scored,
                      'AwayMeanGoalsConceded': away_mean_conceded},
                     dtype=float)

result = grouped.apply(calc_mean_goals).reset_index()
df_worldcup.update(result)


In [37]:
df_worldcup.sort_values(by=['Year', 'HomeTeam'])

,Year,HomeTeam,HomeGoalsScored,HomeGoalsConceded,AwayTeam,AwayGoalsConceded,AwayGoalsScored,Outcome,HomeMeanGoalsScored,HomeMeanGoalsConceded,AwayMeanGoalsScored,AwayMeanGoalsConceded
18,1930.0,Argentina,3,1,Chile,3,1,win,3.0,1.0,1.0,3.0
26,1930.0,Argentina,2,1,France,2,1,win,1.0,0.0,0.0,1.0
41,1930.0,Argentina,6,5,Mexico,6,5,win,6.0,3.0,3.0,6.0
59,1930.0,Argentina,2,1,United States,2,1,win,6.0,1.0,1.0,6.0
117,1930.0,Brazil,1,1,Bolivia,1,1,draw,4.0,0.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
779,2018.0,Tunisia,0,1,England,0,1,lose,1.0,2.0,2.0,1.0
824,2018.0,Uruguay,0,1,Russia,0,1,lose,3.0,0.0,0.0,3.0
814,2018.0,Uruguay,1,0,France,1,0,win,0.0,2.0,2.0,0.0
822,2018.0,Uruguay,3,2,Portugal,3,2,win,2.0,1.0,1.0,2.0


In [38]:
df_worldcup.to_csv('WC_GsGc_mean.csv')